In [18]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, ExtraTreesRegressor, GradientBoostingRegressor
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import xgboost as xgb
import lightgbm as lgb

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor
from sklearn.ensemble import HistGradientBoostingRegressor

In [19]:
input_data = pd.read_csv('2023_smartFarm_AI_hackathon_dataset.csv')
input_data.shape

(84840, 50)

In [20]:
df = input_data.drop(columns=['frmDist'])
df = df.sort_values(by='date')

X = df[df.drop(columns=['outtrn_cumsum','HeatingEnergyUsage_cumsum']).columns]
Y = df[['outtrn_cumsum','HeatingEnergyUsage_cumsum']]
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [21]:
X_train_np = X_train.values
y_train_np = y_train.values

In [22]:
class SimpleEnsemble(BaseEstimator, RegressorMixin):
    def __init__(self, models):
        self.models = [MultiOutputRegressor(model) for model in models]

    def fit(self, X, y):
        for model in self.models:
            model.fit(X, y)
        return self

    def predict(self, X):
        predictions = np.array([model.predict(X) for model in self.models])
        return np.mean(predictions, axis=0)

model_xgb = xgb.XGBRegressor()
GBoost = GradientBoostingRegressor()
LightGB = lgb.LGBMRegressor()
RF = RandomForestRegressor()
ETR = ExtraTreesRegressor()
DT = DecisionTreeRegressor()
HGBR = HistGradientBoostingRegressor()

models = [GBoost, LightGB, RF, model_xgb, ETR, DT, HGBR]

In [23]:
from sklearn.model_selection import KFold
from sklearn.linear_model import Ridge, Lasso
from sklearn.base import clone
from sklearn.multioutput import MultiOutputRegressor

class StackingModel(BaseEstimator, RegressorMixin):
    def __init__(self, base_models, meta_model, n_folds=5):
        self.base_models = base_models
        self.meta_model = meta_model
        self.n_folds = n_folds

    def fit(self, X, y):
        self.base_models_ = [list() for model in self.base_models]
        self.meta_model_ = clone(self.meta_model)
        kfold = KFold(n_splits=self.n_folds, shuffle=True)

        meta_train_data = np.zeros((X.shape[0], len(self.base_models) * y.shape[1]))

        for i, model in enumerate(self.base_models):
            for train_index, holdout_index in kfold.split(X, y):
                instance = clone(model)
                multi_target_instance = MultiOutputRegressor(instance)
                self.base_models_[i].append(multi_target_instance)
                multi_target_instance.fit(X[train_index], y[train_index])
                y_pred = multi_target_instance.predict(X[holdout_index])
                meta_train_data[holdout_index, i*y.shape[1]:(i+1)*y.shape[1]] = y_pred

        self.meta_model_ = MultiOutputRegressor(self.meta_model_)
        self.meta_model_.fit(meta_train_data, y)
        return self

    def predict(self, X):
        meta_features = np.zeros((X.shape[0], len(self.base_models) * y_train.shape[1]))
        for i, base_models in enumerate(self.base_models_):
            for model in base_models:
                y_pred = model.predict(X)
                meta_features[:, i*y_train.shape[1]:(i+1)*y_train.shape[1]] += y_pred
        meta_features /= self.n_folds
        return self.meta_model_.predict(meta_features)

models.append(Lasso())
model = StackingModel(base_models=models, meta_model=Ridge())

model.fit(X_train.values, y_train.values)
y_pred = model.predict(X_test.values)

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.020480 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 8972
[LightGBM] [Info] Number of data points in the train set: 54297, number of used features: 39
[LightGBM] [Info] Start training from score 25575.015289
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.019840 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 8972
[LightGBM] [Info] Number of data points in the train set: 54297, number of used features: 39
[LightGBM] [Info] Start training from score 144010.129186
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005107 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 8970
[LightGBM] [Info] Number of data point

In [24]:
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2score = r2_score(y_test, y_pred)

print("RMSE:", rmse)
print("R2_score:", r2score)

RMSE: 22306.826904546775
R2_score: 0.9983953453023178
